***

<center><u><H1>Artificial Neural Network: classificazione</H1></u></center>

***

***
# Il dataset

Per avere un confronto di classificazione con Machine Learning, riprendiamo il dataset titanic.csv, fornito su Virtual Learning Environment e disponibile anche a [questo link](https://github.com/datasciencedojo/datasets/blob/master/titanic.csv). Il dataset contiene varie informazioni sui passeggeri del Titanic e, come DV, l'informazione relativa alla sopravvivenza. Le feature sono:
- **PassengerId**: ID del passeggero;
- **Survived**: sopravvivenza indicata come 1 (si) o 0 (no);
- **Pclass**: classe del biglietto (da 1 a 3);
- **Name**: nome del passeggero;
- **Sex**: genere del passeggero;
- **Age**: età del passeggero;
- **SibSp**: numero di fratelli/sorelle/sposi a bordo;
- **Parch**: numero di parenti (figli inclusi) a bordo;
- **Ticket**: numero del biglietto;
- **Fare**: prezzo del biglietto;
- **Cabin**: numero della cabina;
- **Embarked**: porto in cui si è imbarcato il passeggero (C = Cherbourg, Q = Queenstown, S = Southampton).

Importiamo i primi moduli necessari:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("00_datasets/titanic.csv")
df

Andando a stampare i data type con il metodo info(), vediamo quali sono le feature che vogliamo mantenere:

In [ ]:
df.info()

Come data type delle diverse feature, non sembrano esserci delle anomalie. Osserviamo le caratteristiche principali delle feature numeriche e categoriche:

In [ ]:
df.describe()

In [ ]:
df.describe(include='object').T

E' opportuno notare come ci sia una discreta predominanza di passeggeri di sesso maschile (577 su 891) e di passeggeri che non sono sopravvissuti (Surived è considerata numerica, ma con soli valori 1 o 0 ha un valor medio di 0.38).

In [ ]:
p = sns.histplot(df, x="Survived",stat="percent")
p.set_xticks([0, 1])
p.set_xticklabels(['No','Si'])
plt.show()

Spostiamo la DV in fondo al dataset come da prassi:

In [ ]:
survived_col = df.pop("Survived")
df['Survived'] = survived_col
df

## Pre-processing

### Dati mancanti

Prima di tutto, vediamo se ci sono dei dati mancanti. Prima di fare la ricerca dei nan, conviene verificare se le feature categoriche non hanno dei caratteri anomali. Applichiamo questa ricerca a tutte le feature categoriche tranne il nome, che non sarà significativo per la predizione della sopravvivenza:

In [ ]:
df_obj = df.select_dtypes(include=['object'])
df_obj

In [ ]:
df_obj.drop('Name', axis=1, inplace=True)
df_obj

In [ ]:
for col in df_obj.columns:
    print(df[col].unique())
    print("-"*40)

Sul genere dei passeggeri non ci sono problemi; anche per il numero del biglietto non sembrano esserci anomalie. In ogni caso si tratta di una feature non rilevante per la classificazione da implementare. Procediamo quindi all'analisi dei dati mancanti:

In [ ]:
missing_data = df.isnull()

for col in missing_data.columns:
    count = sum(missing_data[col])
    if count > 0:
        print(f"{col}: {count/missing_data.shape[0]:.2%} ({count} su {missing_data.shape[0]}) mancanti.")

Per quanto riguarda il porto di imbarco, poiché l'obiettivo è predire se un passeggero è sopravvissuto oppure no, possiamo ignorare questa informazione scartando la feature: sicuramente non è un dato rilevante sulla sopravvivenza.

L'informazione sulla cabina sarebbe stata probabilmente utile, dato che la numerazione sarebbe legata alla posizione nella nave. Tuttavia, sono davvero molti i dati mancanti e cercare una sostituzione adatta quando l'informazione manca per il 77% delle osservazioni non è una scelta valida (si introdurrebbe un forte bias). Scartiamo quindi anche questa feature.

Per finire, in merito all'età dei passeggeri, abbiamo una discreta percentuale di dati mancanti, ma si tratta di una feature potenzialmente utile. Avendo a disposizione circa l'80% di informazioni, possiamo pensare di utilizzare la media come sostituzione.

In [ ]:
df.drop(['Embarked', 'Cabin'], axis=1, inplace=True)
df

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df[['Age']] = imputer.fit_transform(df[['Age']])
df

Dato che l'obiettivo è la classificazione in base alla sopravvivenza, manteniamo nel dataset solamente delle feature che possono avere un significato da questo punto di vista. Andiamo quindi a rimuovere PassengerId, Name, Ticket, Fare. Per quanto riguarda il prezzo, si potrebbe pensare ad una correlazione con la sopravvivenza in quanto i più costosi implicano una posizione di prima classe; tuttavia, l'informazione sulla classe è già in un'altra feature, per cui non occorre mantenere questa.

In [ ]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Fare'], axis=1, inplace=True)
df

### Outlier

Analizziamo visivamente gli outlier. Ci sono delle feature per le quali il boxplot non ha molto senso in quanto contengono una informazione discreta (Pclass, Survived) oppure in quanto categoriche (Sex). Per queste, andremo a valutare un istogramma per osservare come sono distribuiti i dati:

In [ ]:
sns.boxplot(y='Age', data=df, width=0.3)
plt.show()

Per quanto riguarda l'età, si notano diversi outlier; tuttavia, si tratta di valori ampiamente leciti come età di una persona, per cui non dobbiamo preoccuparci di gestirli per adesso.

In [ ]:
sns.boxplot(y='SibSp', data=df, width=0.3)
plt.show()

Alcuni outlier di questa feature arrivano fino ad 8; questa variabile indica il numero di fratelli/sorelle/sposi a bordo. Considerando l'epoca, non è da escludersi che vi siano persone con questo numero di parentele di primo grado.

In [ ]:
sns.boxplot(y='Parch', data=df, width=0.3)
plt.show()

Da qui notiamo che la maggior parte dei passeggeri hanno viaggiato senza genitori o figli (da notare che alcuni bambini viaggiavano con la tata, per cui questo valore corrisponde a 0 per loro). Data questa predominanza, le persone con genitori o più figli a bordo risultano negli outlier, ma non sembrano numeri impossibili.

Osserviamo gli istrogrammi delle altre feature:

In [ ]:
df['Pclass'].value_counts().plot(kind='bar')

Possiamo notare come la maggior parte dei passeggeri abbia viaggiato in terza classe, mentre prima e seconda classe hanno una popolazione simile.

In [ ]:
df['Sex'].value_counts().plot(kind='bar')

Notiamo come il rapporto tra passeggeri di sesso maschile e femminile sia di circa 2 a 1.

La feature Survived è stata già osservata precedentemente. Vediamo anche gli istogrammi delle feature relative alla parentela, per comprendere meglio il loro andamento rispetto ai box plot riportati sopra.

In [ ]:
df['SibSp'].value_counts().plot(kind='bar')

In [ ]:
df['Parch'].value_counts().plot(kind='bar')

### Trasformazione

Non ci sono delle feature per cui occorra cambiare l'unità di misura. Tuttavia, abbiamo una feature categorica da convertire. Questa è di tipo binario e con un ordinamento non richiesto, per cui possaimo procedere con il metodo get_dummies(). Inoltre, sarà sufficiente mantenere solo una delle due colonne: se un passeggero non è maschile, allora è femminile! 

In [ ]:
df = df.join(pd.get_dummies(df['Sex']))
df

In [ ]:
df.drop(['Sex', 'male'], axis=1, inplace=True)
survived_col = df.pop("Survived")
df['Survived'] = survived_col
df

Può essere utile osservare la matrice di correlazione e calcolare i P-value per capire quali feature possono essere più utili:

In [ ]:
from scipy import stats

for col in df.columns:
    pearson_coef, p_value = stats.pearsonr(df[col], df['Survived'])
    print(f"[{col}]: corr = {pearson_coef} | p = {p_value}")

Non notiamo dei valori di correlazione significativi...l'unico parzialmente rilevante è female, con un P-value molto basso, seguito con una debole correlazione da Pclass, anch'essa con un P-value basso. Le altre feature sono scarsamente correlate e con una rilevanza statistica della correlazione un po' limitata. Vediamo però se una rete neurale riesce a venirne a capo!

Per eventuali analisi future, possiamo anche salvare il dataframe ottenuto a valle del pre-processing:

In [ ]:
df.to_csv('00_datasets/titanic_proc.csv', index=False)

# Rete neurale

Procediamo in modo simile alla regressione. Abbiamo un numero leggermente maggiore di feature in ingresso, per cui potremo valutare diverse combinazioni...una grid search sarà sicuramente fondamentale.

## Train/test split e feature scaling

Separiamo training e test set con un metodo holdout, per il momento. Lavoriamo con una divisione al 80%-20%, come fatto con la regressione lineare multipla.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

Per quanto riguarda il feature scaling, lo applichiamo a tutte le feature in ingresso (se ci fossero state delle dummy variable, lo avremmo applicato anche a loro):

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Salviamo il numero di feature in una variabile per comodità nella creazione della rete neurale:

In [ ]:
n_feature = X_train.shape[1]
n_feature

Importiamo TensorFlow per poter visualizzare la rete neurale tra poco:

In [ ]:
import tensorflow as tf

## Design della rete con Grid Search

Importiamo i moduli **models** e **layers** da Keras, che occorrono per creare qualsiasi rete neurale; inoltre, importiamo anche la funzione f1_score per calcolare questa metrica e confrontarla con quanto ottenuto con le tecniche di classificazione utilizzate precedentemente (KNN, DT, SVM).

In [ ]:
from keras import models, layers
from sklearn.metrics import f1_score
my_seed = 15

## Architettura

Procediamo con la grid search per ottenere la migliore configurazione in termini di hidden layer e nodi per hidden layer.

Creiamo una funzione che prenda in ingresso il numero di hidden layer e di neuroni per layer. Ricordiamo di indicare la sigmoide come funzione di attivazione dell'output layer. Per quanto riguarda l'ottimizzatore, usiamo ancora Adam ma ricorrendo alla [binary cross-entropy](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a). Inoltre, possiamo anche inserire l'accuratezza come ulteriore metrica da monitorare.

In [ ]:
def create_nn(n_hidden: int, n_neuron: int):
    """
    Questa funzione crea una rete neurale con un numero di hidden layer e di nodi variabile.
    :param n_hidden: numero di hidden layer 
    :param n_neuron: numero di neuroni per hidden layer
    :return: modello di rete neurale compilato con ottimizzatore 'adam' e loss function 'mse'
    """
    tf.keras.utils.set_random_seed(my_seed)
    # layout della rete: input layer
    inputs = layers.Input(name = 'input_layer', shape=(n_feature,) )
    # layout della rete: hidden layer
    hidden = [layers.Dense(name='hidden_layer_1', units=n_feature, activation='relu')(inputs)]
    for n in range(2, n_hidden+1):
        hidden.append(layers.Dense(name = f'hidden_layer_{n}', units = n_neuron, activation = 'relu')(hidden[n-2]))
    # layout della rete: output layer
    outputs = layers.Dense(name = 'output_layer', units = 1, activation = 'sigmoid')(hidden[-1])
    # creazione della rete
    model = models.Model(inputs = inputs, outputs = outputs, name = 'NN_model')
    # compilazione
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    # ritorno 
    return model

Per la classificazione, importiamo il wrapper [KerasClassifier](https://adriangb.com/scikeras/stable/generated/scikeras.wrappers.KerasClassifier.html):

In [ ]:
from scikeras.wrappers import KerasClassifier
model = KerasClassifier(model=create_nn, batch_size=32, epochs=400, verbose=0, n_hidden=1, n_neuron=2)

Definiamo i parametri della ricerca:

In [ ]:
param_grid = {
    'n_hidden': [1, 2, 4, 6],
    'n_neuron': [3, 4, 5]
}

Creiamo l'oggetto GridSearchCV:

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, error_score=0.0, scoring='f1_micro', verbose=0, n_jobs=-1)

Applichiamo il feature scaling e lanciamo la grid search:

In [ ]:
sc = StandardScaler()
X_copy = sc.fit_transform(X)
grid_result = grid_search.fit(X_copy, y, verbose=0)

Visualizziamo il modello della rete neurale appena creata:

In [ ]:
grid_result.best_params_

Addestriamo la rete:

In [ ]:
grid_result.best_score_

Abbiamo ottenuto un buon risultato, consideerando che abbiamo ancora a disposizione il numero delle epoche e l'ottimizzatore come opzioni! Salviamo il modello come il migliore disponibile, per il momento.

In [ ]:
best_model = grid_result.best_estimator_

## Epoche e batch

Ora passiamo all'ottimizzazione di epoche e batch size:

In [ ]:
param_grid = {
    'epochs': [100, 200, 400],
    'batch_size': [16, 32, 64]
}

In [ ]:
model = KerasClassifier(model=create_nn, batch_size=32, epochs=400, verbose=0, n_hidden=2, n_neuron=3)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, error_score=0.0, scoring='f1_micro', verbose=0, n_jobs=-1)

In [ ]:
sc = StandardScaler()
X_copy = sc.fit_transform(X)
grid_result = grid_search.fit(X_copy, y, verbose=0)

In [ ]:
grid_result.best_params_

In [ ]:
grid_result.best_score_

Altre soluzioni si possono ottenere chiaramente valutando altri iper-parametri, ma anche con una ricerca su tutti i valori contemporaneamente come già indicato nella regressione con rete neurale.